In [5]:
!pip install pinecone-client sentence-transformers pandas# Install latest Pinecone SDK (rename: pinecone), sentence‑transformers, pandas
!pip install --upgrade pip
!pip install --quiet \
    "pinecone[grpc]" \
    sentence-transformers \
    pandas

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `pip install pinecone-client sentence-transformers pandas# Install latest Pinecone SDK (rename: pinecone), sentence‑transformers, pandas'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer

from pinecone import Pinecone, ServerlessSpec, CloudProvider, AwsRegion

# Set your Pinecone API key (you can also set via Colab secrets or env var)
os.environ["PINECONE_API_KEY"] = ""
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

assert PINECONE_API_KEY, "Please set your Pinecone API key"

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Select embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')
embedding_dim = model.get_sentence_embedding_dimension()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
INDEX_NAME = "fineweb-index-2025"
DIMENSION = embedding_dim
METRIC = "cosine"  # or "dotproduct", "euclidean"

# Delete existing index if present
if pc.has_index(name=INDEX_NAME):
    print(f"Deleting existing index: {INDEX_NAME}")
    pc.delete_index(name=INDEX_NAME)

# Create new index
spec = ServerlessSpec(cloud=CloudProvider.AWS, region=AwsRegion.US_EAST_1)
pc.create_index(
    name=INDEX_NAME,
    dimension=DIMENSION,
    metric=METRIC,
    spec=spec
)
print(f"Created index: {INDEX_NAME}")

# Connect to the index object
index = pc.Index(name=INDEX_NAME)

Created index: fineweb-index-2025


In [9]:
# Example: assume a CSV file with a column "text"
file_path = "/content/customers-100.csv"  # replace with actual path or upload

df = pd.read_csv(file_path)
print("Dataset sample:")
print(df.head())

# Ensure the column exists
assert 'text' in df.columns, "Dataset must contain a 'text' column"

Dataset sample:
     text
0  Baxter
1  Lozano
2   Berry
3   Olsen
4  Bender


In [10]:
batch_size = 100
vectors = []

for idx, row in df.iterrows():
    text = str(row['text'])
    vec = model.encode(text).tolist()
    vectors.append((str(idx), vec, {"text": text}))

    if len(vectors) >= batch_size:
        index.upsert(vectors=vectors)
        print(f"Upserted batch ending at row {idx}")
        vectors = []

# Upsert remaining
if vectors:
    index.upsert(vectors=vectors)
    print("Upserted final batch")

print("Upsert complete. Total records:", index.describe_index_stats())

Upserted batch ending at row 99
Upsert complete. Total records: {'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 100}},
 'total_vector_count': 100,
 'vector_type': 'dense'}


In [12]:
# Create example.txt in Colab
example_texts = [
    "Alice loves shopping at local stores",
    "Bob enjoys reading books on history",
    "Carol is interested in gourmet cooking",
    "David likes to travel internationally",
    "Eve works in software engineering"
]

with open("example.txt", "w", encoding="utf-8") as f:
    for line in example_texts:
        f.write(line + "\n")

print("example.txt created!")

example.txt created!


In [14]:
text_file = "example.txt"  # one sentence per line, replace with actual path

with open(text_file, "r", encoding="utf‑8") as f:
    lines = [l.strip() for l in f if l.strip()]

vectors = []
for i, line in enumerate(lines):
    vec = model.encode(line).tolist()
    vectors.append((f"text_{i}", vec, {"text": line}))

    if len(vectors) >= batch_size:
        index.upsert(vectors=vectors)
        print(f"Upserted text‑file batch up to {i}")
        vectors = []

if vectors:
    index.upsert(vectors=vectors)
    print("Final upload from text file complete")

print("Updated stats:", index.describe_index_stats())

Final upload from text file complete
Updated stats: {'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 100}},
 'total_vector_count': 100,
 'vector_type': 'dense'}


In [17]:
query_text = "Why are you Gay"
query_vec = model.encode(query_text).tolist()

top_k = 5
results = index.query(
    vector=query_vec,
    top_k=top_k,
    include_metadata=True
)

print("Query results:")
for match in results['matches']:
    print(f"ID: {match['id']}  Score: {match['score']:.4f}  Text: {match['metadata']['text']}")


Query results:
ID: 37  Score: 0.7030  Text: Gay
ID: 75  Score: 0.2982  Text: Riddle
ID: 93  Score: 0.2849  Text: Young
ID: 69  Score: 0.2785  Text: Good
ID: 53  Score: 0.2586  Text: Richard
